### Name: Shail Shah
### GitHub: anonshail
### USC ID: 9975958845

#### Imports

In [2]:
import pandas as pd
from collections import defaultdict
import os
import numpy as np
import warnings
from sklearn.utils import resample
import glob
import re
warnings.filterwarnings('ignore')

#### 1.a Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29. The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity. 1 Each file containis 6 time series collected from activities of the same person, which are called avg_rss12, var_rss12, avg_rss13, var_rss13, vg_rss23, and ar_rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

#### 1.b Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

Certain files had to be manually cleaned for this assignement
- dataset 9 and 14 had a trailin comma at EOF
- dataset 4 is space seperated not comma seperated

The training data can be found in `../data/train`
The testing data can be found in `../data/train`

#### 1.c Feature Extraction
#### Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

#### 1.c.i Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

Type of time-domain features that are usually used in time series classification are as follows:
- Minimum value of the time series
- Maximum value of the time series
- Mean value of the time series
- Median value of the time series
- Coefficient of Skewness
- Coefficient of Kurtosis
- Standard deviation of the time series
- Range of the time series
- Cutoregressive (AR) part of the model
- Integrated (I) part of the model
- Moving Average (MA) part of the model

#### 1.c.ii Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.


In [3]:
cols = ["avg_rss12","var_rss12","avg_rss13", "var_rss13","avg_rss23", "var_rss23"]
dirs = [
    "bending1",
    "bending2",
    "cycling", 
    "lying",
    "sitting",
    "standing", 
    "walking"
]
#data set
ds = defaultdict(list)


for d in dirs:
    for file in os.listdir(f'../data/test/{d}'):
        if(not file.endswith('.csv')):
            continue
        dfi = pd.read_csv(f'../data/test/{d}/{file}', skiprows=4, usecols=range(0,7))
        for c in cols:
            x = pd.DataFrame(dfi[c])
            
            # "mean","min","max","median","first_quart","third_quart","std_dev"
            ds[f'{c}_min'].append(np.min(x)[c])
            ds[f'{c}_max'].append(np.max(x)[c])
            ds[f'{c}_mean'].append(np.mean(x)[c])
            ds[f'{c}_median'].append(np.median(x))
            ds[f'{c}_first_quart'].append(np.quantile(x,0.25))
            ds[f'{c}_third_quart'].append(np.quantile(x,0.75))
            ds[f'{c}_std_dev'].append(np.std(x)[c])

for d in dirs:
    for file in os.listdir(f'../data/train/{d}'):
        if(not file.endswith('.csv')):
            continue
        dfi = pd.read_csv(f'../data/train/{d}/{file}', skiprows=4, usecols=range(0,7))
        for c in cols:
            x = pd.DataFrame(dfi[c])
            
            # "mean","min","max","median","first_quart","third_quart","std_dev"
            ds[f'{c}_min'].append(np.min(x)[c])
            ds[f'{c}_max'].append(np.max(x)[c])
            ds[f'{c}_mean'].append(np.mean(x)[c])
            ds[f'{c}_median'].append(np.median(x))
            ds[f'{c}_first_quart'].append(np.quantile(x,0.25))
            ds[f'{c}_third_quart'].append(np.quantile(x,0.75))
            ds[f'{c}_std_dev'].append(np.std(x)[c])


df = pd.DataFrame.from_dict(ds)           
df

    

,avg_rss12_min,avg_rss12_max,avg_rss12_mean,avg_rss12_median,avg_rss12_first_quart,avg_rss12_third_quart,avg_rss12_std_dev,var_rss12_min,var_rss12_max,var_rss12_mean,...,avg_rss23_first_quart,avg_rss23_third_quart,avg_rss23_std_dev,var_rss23_min,var_rss23_max,var_rss23_mean,var_rss23_median,var_rss23_first_quart,var_rss23_third_quart,var_rss23_std_dev
0,38.00,45.67,42.812812,42.500,42.0000,43.6700,1.434054,0.0,1.22,0.372438,...,32.0000,34.50,1.993175,0.0,3.11,0.571083,0.430,0.0000,1.3000,0.600383
1,37.25,45.00,40.624792,40.500,39.2500,42.0000,1.475428,0.0,1.30,0.358604,...,33.0000,36.00,2.186168,0.0,1.92,0.570583,0.430,0.0000,1.3000,0.582308
2,0.00,42.75,27.464604,28.000,25.5000,30.0000,3.579847,0.0,7.76,0.449708,...,15.0000,20.75,5.048375,0.0,6.76,1.122125,0.830,0.4700,1.3000,1.011287
3,12.75,51.00,24.562958,24.250,23.1875,26.5000,3.733619,0.0,6.87,0.590833,...,20.5000,27.00,3.689936,0.0,4.97,0.700188,0.500,0.4300,0.8700,0.692997
4,28.75,44.75,37.561188,36.875,35.2500,40.2500,3.223144,0.0,9.91,2.080688,...,18.0000,21.50,2.724534,0.0,9.62,2.765896,2.450,1.4100,3.7700,1.767359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,21.50,51.00,34.935813,35.500,32.0000,38.0625,4.641102,0.0,12.21,4.115750,...,14.2375,18.25,3.188731,0.0,10.21,3.280021,3.015,2.1200,4.5000,1.699145
84,18.00,46.00,35.193333,36.000,32.0000,38.7500,4.746916,0.0,16.20,4.321021,...,14.2500,18.50,3.204299,0.0,8.50,3.241958,3.015,1.8850,4.4400,1.767339
85,19.75,45.50,34.322750,35.250,31.0000,38.0000,4.747524,0.0,13.47,4.456333,...,13.5000,17.75,3.116605,0.0,9.67,3.432562,3.200,2.1575,4.5650,1.730921
86,21.50,51.25,34.253521,35.000,30.9375,37.7500,4.935592,0.0,13.55,4.457896,...,13.7500,18.00,3.113379,0.0,8.32,3.500750,3.285,2.1800,4.5575,1.690614


#### 1.c.iii Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [4]:
standard = pd.DataFrame(df.std())
standard.columns = ["Standart Deviation"]

conflb = {}
confub = {}
for c in df.columns:
    b = []
    for i in range(500):
        subset = resample(df[c],n_samples=88)
        b.append(np.std(subset))
    b.sort()
    conflb[c]=b[24]    #10% of 500
    confub[c]=b[474]   #90% of 500
    
lbdf = pd.DataFrame.from_dict(conflb, orient='index')
lbdf.columns = ['Conf Interval Lower Bound']

ubdf = pd.DataFrame.from_dict(confub, orient='index')
ubdf.columns = ['Conf Interval Upper Bound']

ans = standard.join(lbdf)
ans = ans.join(ubdf)
ans

,Standart Deviation,Conf Interval Lower Bound,Conf Interval Upper Bound
avg_rss12_min,9.569975,8.044849,10.618680
avg_rss12_max,4.394362,3.323935,5.249531
avg_rss12_mean,5.335718,4.695083,5.848076
avg_rss12_median,5.440054,4.775468,5.960260
avg_rss12_first_quart,6.153590,5.493338,6.583586
avg_rss12_third_quart,5.138925,4.250853,5.807053
avg_rss12_std_dev,1.770306,1.557450,1.932606
var_rss12_min,0.000000,0.000000,0.000000
var_rss12_max,5.062729,4.608276,5.363823
var_rss12_mean,1.574164,1.389409,1.688498


#### 1.c.iv Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

- Median: This will give us an idea of the central tendancy of the dataset, which can help with dealing with outliers.

- Mean: This will give an aggregated value of the dataset. We can also compare the mean with the media to understand how the data is distributed from the lowest to the highest value. If the value of the mean and median are about the same, then the dataset is evenly distributed.

- Maximum value: This will give us an idea of the upper limit value of the time series.


#### 2. I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 + β1X + β2X2 + β3X3 + ϵ.

#### 2.a Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

We would expect the cubic model to have a lower RSS value than the linear model on the training data as the cubic model would fit better as it is more flexible.

#### 2.b Answer (a) using test rather than training RSS.

As we know the relation between X and Y is linear, RSS value for the cubic model would be greater than the linear model for test data.

#### 2.c Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

I would expect the RSS value of the cubic regression to be lesser than the linear regression on the training data since cubic model is more flexible.

#### 2.d Answer (c) using test rather than training RSS.

More information is required for this case.  If the true relationship is closer to linear than cubic, the linear model would do better. However, if the relation is closer to the cubic, then the cubic model would perform better. 